# Images range adjustment

This notebook is designed to calculate the range adjustment that needs to be applied to CT and PET images.

In [ ]:
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import numpy as np

import env_apps


## Images extraction

We first extract the prostate pixel values from all the images of the training patients.

We define the `patients_data_extractor`.

In [ ]:
from delia.databases import PatientsDatabase
from delia.extractors import PatientsDataExtractor
from delia.transforms import (
    MatchingCentroidSpatialCropD,
    MatchingCropForegroundD,
    MatchingResampleD,
    PETtoSUVD,
    ResampleD
)
from monai.transforms import (
    CenterSpatialCropD,
    Compose,
    KeepLargestConnectedComponentD,
    ScaleIntensityD,
    SpatialCropD,
    ThresholdIntensityD,
)


env_apps.configure_logging("logging_conf.yaml")

transforms = Compose(
    [
        ResampleD(keys=["CT"], out_spacing=(1.0, 1.0, 1.0)),
        MatchingResampleD(reference_image_key="CT", matching_keys=["PT", "Prostate"]),
        MatchingCropForegroundD(reference_image_key="CT", matching_keys=["PT", "Prostate"]),
        SpatialCropD(keys=["CT", "PT", "Prostate"], roi_slices=[slice(30, 740), slice(None), slice(None)]),
        KeepLargestConnectedComponentD(keys=["Prostate"]),
        MatchingCentroidSpatialCropD(segmentation_key="Prostate", matching_keys=["CT", "PT"], roi_size=(128, 128, 128)),
        PETtoSUVD(keys=["PT"]),
    ]
)

patients_data_extractor = PatientsDataExtractor(
    path_to_patients_folder=r"local_data/Learning_set",
    series_descriptions=r"local_data/series_descriptions.json",
    transforms=transforms
)


We extract the patients' data. This step takes a while.

In [ ]:
CT_pixel_values_in_all_patients_roi = []
PT_pixel_values_in_all_patients_roi = []
for i, patient_data in enumerate(patients_data_extractor):
    print(f"{'-'*20}\nPatient ID: {patient_data.patient_id}")

    for patient_image_data in patient_data.data:
        dicom_header = patient_image_data.image.dicom_header
        numpy_array_image = patient_image_data.image.numpy_array
        
        if dicom_header.Modality == "CT":
            ct_array = numpy_array_image
        if dicom_header.Modality == "PT":
            pt_array = numpy_array_image

        segmentations = patient_image_data.segmentations
        if segmentations:
            for organ, numpy_array_image in segmentations[0].numpy_array_label_maps.items():
                if organ == "Prostate":
                    prostate_mask_array = numpy_array_image
                    
    ct_roi = ct_array[np.where(prostate_mask_array)]
    pt_roi = pt_array[np.where(prostate_mask_array)]
    
    CT_pixel_values_in_all_patients_roi.extend(ct_roi)
    PT_pixel_values_in_all_patients_roi.extend(pt_roi)


We copy the data to avoid overwriting it.

In [ ]:
ct_pixels = deepcopy(CT_pixel_values_in_all_patients_roi)
pt_pixels = deepcopy(PT_pixel_values_in_all_patients_roi)

## CT range adjustment

We follow the methodology described in:

> Shahedi M, Halicek M, Guo R, Zhang G, Schuster DM, Fei B. A semiautomatic segmentation method for prostate in CT images using local texture classification and statistical shape modeling. Med Phys. 2018 Jun;45(6):2527-2541. doi: 10.1002/mp.12898. Epub 2018 Apr 23. PMID: 29611216; PMCID: PMC6149529.

The voxels with HU values below -200 and above 250 were assigned HU values of -200 and 250, respectively. We determined these HU threshold levels by observing the HU range for prostate voxels across the training images and removing the 0.2% outliers. The resulting HU values ranged from -178 to 244, so we arbitrarly selected the -200 to 250 range as a slightly larger range.

First, we calculate the HU range after removing the outliers

In [37]:
ct_pixels = np.array(ct_pixels)

def reject_outliers(data, m=5.5869):
    filtering = np.absolute(data - np.mean(data)) < m * np.std(data)
    return data[filtering]

cleaned_ct_pixels = reject_outliers(ct_pixels)

print(f"Percentage of outliers removed: {(1 - len(cleaned_ct_pixels)/len(ct_pixels))*100} %")
print(f"Range: [{np.min(cleaned_ct_pixels)}, {np.max(cleaned_ct_pixels)}] HU")
print(f"Mean: {np.mean(cleaned_ct_pixels)} HU")
print(f"Median: {np.median(cleaned_ct_pixels)} HU")


Percentage of outliers removed: 0.19983215275543031 %
Range: [-178, 244] HU
Mean: 32.52880449817627 HU
Median: 33.0 HU


We then clip the pixels' values to our arbirary range (based on the range obtained with the previous step).

In [38]:
clipped_ct_pixels = np.clip(ct_pixels, -200, 250)
number_of_outliers = len(ct_pixels[(ct_pixels < -200) | (ct_pixels > 250)])

print(f"Percentage of outliers removed: {(number_of_outliers/len(ct_pixels))*100} %")
print(f"Range: [{np.min(clipped_ct_pixels)}, {np.max(clipped_ct_pixels)}] HU")
print(f"Mean: {np.mean(clipped_ct_pixels)} HU")
print(f"Median: {np.median(clipped_ct_pixels)} HU")

Percentage of outliers removed: 0.18646426134334654 %
Range: [-200, 250] HU
Mean: 32.85776447682485 HU
Median: 34.0 HU


We compute the global CT images histogram.

In [ ]:
plt.hist(
    ct_pixels, 
    bins=25, 
    weights=np.ones(len(ct_pixels)) / len(ct_pixels)
)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.show()

plt.hist(
    ct_pixels, 
    bins=25, 
    weights=np.ones(len(ct_pixels)) / len(ct_pixels)
)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.yscale('log')
plt.show()


We compute the clipped CT images histogram.

In [ ]:
plt.hist(
    clipped_ct_pixels, 
    bins=25, 
    weights=np.ones(len(clipped_ct_pixels)) / len(clipped_ct_pixels)
)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.show()

plt.hist(
    clipped_ct_pixels, 
    bins=25, 
    weights=np.ones(len(clipped_ct_pixels)) / len(clipped_ct_pixels)
)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.yscale('log')
plt.show()



## PT range adjustment

To ensure consistency and avoid potential inaccuracies, we applied a threshold of 25 to the Standardized Uptake Value (SUV) for all voxels with SUV values above this limit. This threshold was determined based on the maximum SUV value (24.9) observed in our training images, as determined by the radiologist who reviewed our imaging data. By capping the SUV values above 25, we are able to maintain a standardized approach that reduces the impact of any variability in tracer uptake, and ensures a more reliable and reproducible assessment of metabolic activity in our imaging data.

We clip the pixels' values to our arbirary range.

In [41]:
pt_pixels = np.array(pt_pixels)

clipped_pt_pixels = np.clip(pt_pixels, 0, 25)
number_of_outliers = len(pt_pixels[(pt_pixels < 0) | (pt_pixels > 25)])

print(f"Percentage of outliers removed: {(number_of_outliers/len(pt_pixels))*100} %")
print(f"Range: [{np.min(clipped_pt_pixels)}, {np.max(clipped_pt_pixels)}] SUV")
print(f"Mean: {np.mean(clipped_pt_pixels)} SUV")
print(f"Median: {np.median(clipped_pt_pixels)} SUV")


Percentage of outliers removed: 0.06613541477938033 %
Range: [0.0, 25.0] SUV
Mean: 2.4371612121191992 SUV
Median: 2.1341411159528114 SUV


We compute the global PT images histogram.

In [ ]:
plt.hist(
    pt_pixels,
    bins=25,
    weights=np.ones(len(pt_pixels)) / len(pt_pixels)
)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.show()

plt.hist(
    pt_pixels, 
    bins=25, 
    weights=np.ones(len(pt_pixels)) / len(pt_pixels)
)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.yscale('log')
plt.show()


We compute the clipped PT images histogram.

In [ ]:
cleaned_pt_pixels = np.clip(pt_pixels, 0, 25)

plt.hist(
    cleaned_pt_pixels, 
    bins=25, 
    weights=np.ones(len(cleaned_pt_pixels)) / len(cleaned_pt_pixels)
)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.show()

plt.hist(
    cleaned_pt_pixels, 
    bins=25, 
    weights=np.ones(len(cleaned_pt_pixels)) / len(cleaned_pt_pixels)
)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.yscale('log')
plt.show()